## Validation of perceptron

In [24]:
%load_ext autoreload
%autoreload 2
import random
from perceptron import Perceptron
import numpy as np
from sklearn.datasets import load_digits

from sklearn.model_selection import train_test_split


X, y = load_digits(return_X_y=True, n_class=2)

# Split in 60/40
Xtrain, Xdata, yTrain, yData = train_test_split(X, y * 2 - 1, test_size=0.2, random_state=42, stratify=(y * 2 - 1))
# Split in 50/50 the remaining 40%
Xval, Xtest, yVal, yTest = train_test_split(Xdata, yData, test_size=0.5, random_state=42, stratify=yData)
np.random.seed(42)
accuracies = []
for i in range(100):
    initial_weights = np.random.randn(Xtrain.shape[1] + 1)
    for alpha in [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10, 10**1, 10**2, 10**3, 10**4]:
        perceptron = Perceptron(alpha)
        perceptron.train(Xtrain, yTrain, initial_weights)
        predictions = perceptron.predict(Xval)
        accuracy = np.mean(predictions == yVal)
        margin = perceptron.compute_min_margin(Xval)
        accuracies.append((initial_weights, alpha, accuracy, margin))
        
# Find the maximum accuracy
max_accuracy = max(accuracies, key=lambda x: x[2])[2]

# Filter accuracies based on the maximum accuracy
best = [el for el in accuracies if el[2] == max_accuracy]

for el in best:
    print(f'Alpha: {el[1]} Accuracy: {el[2]} Margin: {el[3]}')

best_margin = max(best, key=lambda x: x[3])
print(f'Initial_weights: {best_margin[0]} Alpha: {best_margin[1]} Accuracy: {best_margin[2]} Margin: {best_margin[3]}')

# save weights and alpha in a file
np.save('weights', best_margin[0])
np.save('alpha', best_margin[1])





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Alpha: 0.1 Accuracy: 1.0 Margin: 6.06971954330057
Alpha: 100 Accuracy: 1.0 Margin: 2.1688291732164746
Alpha: 1000 Accuracy: 1.0 Margin: 2.1783096065226557
Alpha: 10000 Accuracy: 1.0 Margin: 2.179257576200012
Alpha: 0.1 Accuracy: 1.0 Margin: 3.31792508384932
Alpha: 100 Accuracy: 1.0 Margin: 2.175241631305046
Alpha: 1000 Accuracy: 1.0 Margin: 2.178950843901812
Alpha: 10000 Accuracy: 1.0 Margin: 2.179321699852448
Alpha: 0.01 Accuracy: 1.0 Margin: 0.33895687300425337
Alpha: 0.1 Accuracy: 1.0 Margin: 2.0310712070567205
Alpha: 0.01 Accuracy: 1.0 Margin: 0.5402782803754356
Alpha: 10 Accuracy: 1.0 Margin: 2.185369687196407
Alpha: 10 Accuracy: 1.0 Margin: 2.185369687196407
Alpha: 100 Accuracy: 1.0 Margin: 2.1799691358418034
Alpha: 1000 Accuracy: 1.0 Margin: 2.1794235836248923
Alpha: 10000 Accuracy: 1.0 Margin: 2.179368973716274
Alpha: 0.001 Accuracy: 1.0 Margin: 0.18806464181395
Alpha: 0.01 Accuracy: 1.0 Mar

## Testing the perceptron on the test set

In [27]:
# load weights and alpha from a file
weights = np.load('weights.npy')
alpha = np.load('alpha.npy')
# predict on test set
perceptron = Perceptron(alpha)
perceptron.train(Xtrain, yTrain, weights)
predictions = perceptron.predict(Xtest)
accuracy = np.mean(predictions == yTest)
print(f'Accuracy on test set: {accuracy}')
print(f'Margin on test set: {perceptron.compute_min_margin(Xtest)}')

perceptron = Perceptron(alpha)
perceptron.train(Xtrain, yTrain, weights)
predictions = perceptron.predict(Xval)
accuracy = np.mean(predictions == yVal)
print(f'Accuracy on eval set: {accuracy}')
print(f'Margin on eval set: {perceptron.compute_min_margin(Xval)}')

Accuracy on test set: 1.0
Margin on test set: 5.455034884107503
Accuracy on eval set: 1.0
Margin on eval set: 6.06971954330057
